# 10 - Advanced topics: High Performance Computing Examplel

    
###    STILL UNDER CONSTRUCTION
    
This allocates the day_dates generated to the different codes in as many nodes are available. 
Works inside and outside of slurm for testing (but set FullYear to False so it only does two days)
Full year takes 1 min in 11 Nodes.

    Variables stored in input_bf.py. First configure this on top:
    
    .. code-block :: python
    
        if __name__ == "__main__": #in case this is run as a script not a module.
            from readepw import readepw  
            from load import loadTrackerDict
            from input_bf import *

        else: # module imported or loaded normally
            from bifacial_radiance.readepw import readepw # epw file reader from pvlib development forums  #module load format
            from bifacial_radiance.load import loadTrackerDict
            from bifacial_radiance.input_bf import *
                
    Procedure for a Full Year Run (~1 min in 11 nodes of 36 cores each > 365 days):

    .. code-block :: python

        Connect to Eagle
        - $ cd bifacial_radiance/bifacial_radiance
        - $ srun -A pvsoiling -t 5 -N 11 --pty bash                  
        - $ module load conda
        - $ . activate py3
        - $ srun bifacial_radiance2.py
    
    Procedure for testing before joining SLURM:
    
    .. code-block :: python

        change fullYear to False.
        - $ cd bifacial_radiance/bifacial_radiance
        - $ module load conda
        - $ . activate py3
        - $ nano bifacial_radiance.py    
        - $ python bifacial_radiance2.py       
    
    .. warning::
        Do not load conda twice nor activate .py3 twice.
        (following above) Either activate conda or .py3 in the login node
        or on the slurm

In [9]:
from bifacial_radiance import *
import multiprocessing as mp
import datetime
import os

### INPUT FILE:
To pytest the HPC routine and or try it here in the Jupyter Tutorial, we had to add this next cell with all input variables. 

The way it usually works for us on HPC: We generate automatically input files (.py extension, ie. inputvariablefile.py) 
with all the variables for the multiple permutations of simulations we run,  and then import them at the beginning of our HPC main.py (same as any other main.py, it just has "import inputvariablefile.py) 


In [2]:
testfolder = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests'# os.path.dirname(__file__)  # this folder
epwfile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\USA_CO_Boulder.724699_TMY2.epw'
simulationname='PVMC_0'
moduletype = 'PVPMC_0'
numpanels = 1
torqueTube = False
tubetype='Round'
pitch = 8 # gcr - 0.25
hub_height = 2.0
limit_angle = 60 
diameter = 0.1
xgap=0.01 
ygap=0.0
zgap = 0.05
sensorsy=100
x= 1
y = 2
nMods=20
nRows=7
torqueTubeMaterial='Metal_Grey'
backtrack = True
cumulativesky = False
rewriteModule=True
cellLevelModule = False
hpc=True       
axisofrotationTorqueTube=True
gcr = 0.25
albedo = 0.28
modWanted = 10
rowWanted = 4
axis_azimuth=180
angledelta=0.01
roundTrackerAngle= True
bifi=0.9
# starting in v0.4.0, some torque tube parameters are passed separately as a dictionary.
tubeParams = {'diameter':diameter,
              'tubetype':tubetype,
              'material':torqueTubeMaterial,
              'axisofrotation':axisofrotationTorqueTube}

## Run FIRST, before entering Slurm mode 

Multiple Nodes get confused when trying to write the JSON at the same time,
so make sure moduletype is created before running slurm job for it to work.
2 DO: Fix at some point of course.


In [7]:
demo = RadianceObj(simulationname,path=testfolder)
demo.setGround(albedo)
mymodule=demo.makeModule(name=moduletype,x=x,y=y,bifi=bifi,
                           torquetube=torqueTube, zgap = zgap, numpanels = numpanels, ygap = ygap,
                           rewriteModulefile = True, xgap=xgap,
                           tubeParams=tubeParams)
sceneDict = {'pitch': pitch, 'hub_height':hub_height, 'nMods':nMods, 'nRows':nRows}


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests

Module Name: PVPMC_0
Module file did not exist before, creating new module file
Module PVPMC_0 successfully created


## Run on HPC Slurm Mode now: (or your computer with multiple cores. It'll be okay).

Tested on a DELL Laptop, Core i7vPro 8th Gen.

In [10]:
daylist = []

fullYear = True # running faster testing on HPC ~ only 2 days.

if fullYear:
    start = datetime.datetime.strptime("01-01-2014", "%d-%m-%Y")
    end = datetime.datetime.strptime("31-12-2014", "%d-%m-%Y") # 2014 not a leap year.
    daylist.append('12_31')     # loop doesn't add last day. Adding it at the beginning because why not.
    daylimit = 365
else:
    start = datetime.datetime.strptime("14-02-2014", "%d-%m-%Y")
    end = datetime.datetime.strptime("26-02-2014", "%d-%m-%Y") # 2014 not a leap year.
    daylimit = 1
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
for date in date_generated:
    daylist.append(date.strftime("%m_%d"))

demo = RadianceObj(simulationname,path=testfolder)
demo.setGround(albedo)
#   HPC IMPORTANT NOTE:
#   Multiple Nodes get confused when trying to write the JSON at the same time,
#    so make sure moduletype is created before running slurm job for it to work.
#   2 DO: Fix at some point of course.
#    mymodule=demo.makeModule(name=moduletype,x=x,y=y,bifi=bifi,
#                           torquetube=torqueTube, zgap = zgap, numpanels = numpanels, ygap = ygap,
#                           rewriteModulefile = True, xgap=xgap,
#                           tubeParams=tubeParams, cellModule={'numcellsx':numcellsx,'numcellsy': numcellsy) 
#                           
sceneDict = {'pitch': pitch, 'hub_height':hub_height, 'nMods':nMods, 'nRows':nRows}

cores = mp.cpu_count()
pool = mp.Pool(processes=cores)
res = None

try:
    nodeID = int(os.environ['SLURM_NODEID'])
except KeyError: # in case testing for hpc not on slurm yet.
    nodeID = 0 
    print("Seems you are testing outside of slurm or on a reg. computer. It's okay, go on") 

    
hpccores = 36 # this is valid for Eagle. Find out how many cores are in each node of your HPC to make this work.

day_index = (hpccores * (nodeID))

for job in range(cores):
    print ("Assigning", day_index+job)
    if day_index+job>=1: # Short version, running 1 days only.For the year do: len(daylist): # this makes sure no days above 356 are attempted:
        break
    pool.apply_async(bifacial_radiance.main.runJob, (daylist[day_index+job],))  # on the hpc we run this from inside of main.py script, 
                                                              #so we just do "runJob" instead of "main.runJob"
    
pool.close()
pool.join()
pool.terminate()

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests
Seems you are testing outside of slurm or on a reg. computer. It's okay, go on
Assigning 0
Assigning 1


### We are still debugging this part, it's not really going into the runJob function for some reason in this tutorial, but this works for a charm for NREL and SNL......